In [189]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats as st
import seaborn as sns
import math
from statsmodels.stats.multicomp import pairwise_tukeyhsd
import random
from scipy.stats import norm
from tqdm.auto import tqdm

In [190]:
data = pd.read_csv('ba2.csv')

In [191]:
# for i in data.columns:
#     if len(data[i].unique()) > 4:
#         print(f'"{i}",')

In [192]:
TARGET = 'Тяжесть'

In [193]:
data[TARGET].unique()

array([3, 2], dtype=int64)

In [222]:
name_group_all = {
    'Тяжесть' : ['БАТТ', 'БАСТ'],
    'SNP_rs896' : ['TT', 'TC', 'CC']
}

In [223]:
name_group_all[TARGET]

['БАТТ', 'БАСТ']

In [224]:
#обозначение групп

name_group = name_group_all[TARGET]
#spisok_group
data = data.dropna(subset=[TARGET])
data = data.astype({TARGET: np.int32})
#обозначение групп

In [225]:
data[TARGET].unique()

array([3, 2])

In [226]:
#data['Тяжесть'].sum()

In [227]:
#for i in range(len(data['Тяжесть'])):
#    if data.loc[i,'Тяжесть'] == 2:
#        data.loc[i,'Тяжесть'] = 1
#    elif data.loc[i,'Тяжесть'] == 3:
#        data.loc[i,'Тяжесть'] = 0
        

In [228]:
#необходимо обозначить какие переменные к какой группе относятся
scale = [
'Приступов_удушья_в_день',
'Обострений_в_год',
'Длительность_БА_на_момент_исслед',
'САД',
'ДАД',
'ЧД',
'ЧСС',
'Пульс',
'Сатурация_при_поступлении',
'Сатурация_при_выписке',
'Лейкоциты_при_поступлении',
'Эритроциты_при_постпулении',
'Гемоглобин_при_поступлении',
'Тромбоциты_при_поступлении',
'Сегментоядерные_при_поступлении',
'Эозинофилы_при_поступлении',
'Моноциты_при_поступлении',
'Лимфоциты_при_поступлении',
'СОЭ_при_поступлении',
'Лейкоциты_при_выписке',
'Эритроциты_при_выписке',
'Гемоглобин_при_выписке',
'Тромбоциты_при_выписке',
'Сегментоядерные_при_выписке',
'Лимфоциты_при_выписке',
'СОЭ_при_выписке',
'БАК_посев_мокроты',
'IgE_общий',
'FeF25_при_поступлении',
'FeF50_при_поступлении',
'FeF75_при_поступлении',
'FeF25_при_выписке',
'FeF50_при_выписке',
'FeF75_при_выписке',
'ЧСС_ЭКГ',
'Общий_билирубин',
'Глюкоза_крови',
'Общий_белок',
'Холестерин',
'Возраст2',
'Рост',
'Вес',
'обострений_за_последний_год',
'ACQ_контроль_симптомов_БА',
'Самочувствие',
'Активность',
'Настроение',
'Экстраверсия_интроверсия',
'Ригидность_пластичность',
'Эмоционвозбудим_уравновешенность',
'Быстрота_реакц_медлительность',
'Активность_пасивность',
'Искренность',
'Реактивная_тревожность',
'Ситуатвиная_тревожность',
'Физическая_активность_PF',
'Физичпроблемы_в_жизнедеят_RP',
'Интенсивность_боли_BP',
'Общее_здоровье_GH',
'Жизненная_актвиность_VT',
'Социальная_активность_SF',
'Эмоционал_проблемы_в_жизнедеят_RE',
'Психическое_здоровье_MH',
'Физич_компонент_здор_PH',
'Психич_компонент_здор_MH',
'Наличие_вегет_дисфун',
'Наличие_вегет_дисфун2',
'Апатия1',
'Апатия2',
'Уровень_VIP',
'Уровень_SP',
'ЖЕЛ_при_поступлении',
'ОФВ1_при_поступлении',
'Индекс_Тиффно_при_поступлении',
'ЖЕЛ_при_выписке',
'ОФВ1_при_выписке',
'Индекс_Тиффно_при_выписке',
'ИМТ',
'VAR_mc',
'aVNN_mc',
'SDNN_mc',
'pNN50',
'rMSSD',
'SDNNidx_mc',
'SDANN_mc',
'VLF',
'LF_Симпатика',
'HF_Парасимпатика',
'nHF',
'CBBP',
'LF_HF',
]
categorial = [
#"Сезон_обострения_БА",
# "Худшее_время_суток_по_БА",
# "Ухудшение_состояния_в_отпуске",
# "Заболевания_ЖКТ",
# "Заболевания_ЛОРорганов",
# "Терапия_БА",
#"Профессия",
# "Форма",
# "Тяжесть",
#"Оценка_соц_бытовых_условий",
# "ДН",
# "Эмфизема",
# "ХЛС",
# "Пневмосклероз",
# "Гормонозависимость",
# "Проживание",
# "Профессиональное_заболевание",
# "ГБ",
# "ИБС",
# "ГЭРБ",
# "Симбикорт",
# "Формисонид_натив",
# "Дуоресп_Спиромакс",
# "Спирива",
# "Беродуал",
# "Форадил_комби",
# "Сальбутамол",
# "Серетид",
# "Преднизолон_per_os",
# "Антилейкотриеновые",
# "Беклометазон",
# "Эуфиллин_per_os",
# "Сингуляр",
# "Фостер",
# "Сибри",
# "Беротек",
# "Ипратерол",
# "Палочкоядерные_при_поступлении",
# "Базофилы_при_постпулении",
# "Палочкоядерные_при_выписке",
# "Эозинофилы_при_выпсике",
# "Моноциты_при_выписке",
# "Базофилы_при_выписке",
# "Наличие_суносовой_аритмии",
# "Наличие_гипертрофии_ЛЖ",
# "Эозинофилы_в_мокроте",
# "Пол",
# "Наличие_работы",
# "Работа_в_ночную_смену",
# "Профессиональый_контакт",
# "Образование",
# "Наличие_аллергической_реакции",
# "Аллерг_реакция_у_родственников",
# "Наличие_заболевания_ЖКТ",
# "Наличие_заболевания_ЛОРорганов",
# "Наличие_грибковых_заболеваний",
# "Ухудшение_состояния_при_физнагруз",
# "Ухудшение_состояния_во_время_еды",
# "Влияние_менструальногоцикла_на_БА",
# "Наличие_ОтёкаКвинке_в_анамнезе",
# "Наличие_стресса",
# "Наличие_других_заболеваний",
"SNP_rs896",
"SNP_rs2072100",
"SNP_rs3771829",
"SNP_rs3823082",
"Фенотип",
# "Коридор",
# "Симпатика",
]


In [229]:
spisok_group = categorial

In [230]:
def test_fishera(data, i , name_group_all):
    data = data.dropna(subset=[i])
    data = data.astype({i: np.int32})
    n_ind = len(data[i].unique())+1
    data_vr = np.zeros((n_ind, 9))
    col_ch = 0
    for col in data[TARGET].unique():
        vsego = 0
        ind_ch = 0
        for ind in data[i].unique():
            data_vr[ind_ch, col_ch] = data[(data[i] == ind) & (data[TARGET] == col)][TARGET].count()
            vsego += data[(data[i] == ind) & (data[TARGET] == col)][TARGET].count()
            ind_ch += 1
        data_vr[len(data[i].unique()),col_ch] = vsego
        col_ch += 1
    data_vr[n_ind-1,2] = data_vr[n_ind-1,1]
    data_fisher = np.zeros((2,2))   
    for j in range(n_ind-1):
        data_fisher[0,0] = data_vr[j,0]
        data_fisher[0,1] = data_vr[j,1]
        data_fisher[1,0] = data_vr[n_ind-1,0] - data_vr[j,0]
        data_fisher[1,1] = data_vr[n_ind-1,1] - data_vr[j,1]
        x, p = st.fisher_exact(data_fisher)
        data_vr[j,2] = data_fisher[0,1]

        data_vr[j,4] = round(x, 2)
        data_vr[j,5] = round(x, 3)
        data_vr[j,6] = round((data_vr[j,0] / (data_vr[n_ind-1,0] - data_vr[j,0])) /
                             (data_vr[j,1] / (data_vr[n_ind-1,1] - data_vr[j,1])),2)
        data_vr[j,7] = round(math.exp(math.log(data_vr[j,6]) 
                                      - 1.96*math.sqrt(1/data_fisher[0,0]+1/data_fisher[0,1]+1/data_fisher[1,0]+1/data_fisher[1,1])),2)
        data_vr[j,8] = round(math.exp(math.log(data_vr[j,6])
                                               +1.96*math.sqrt(1/data_fisher[0,0]+1/data_fisher[0,1]+1/data_fisher[1,0]+1/data_fisher[1,1])),2)
        data_vr[j,1] = round(data_fisher[0,0] / data_vr[n_ind-1,0] * 100, 2)
        data_vr[j,3] = round(data_fisher[0,1] / data_vr[n_ind-1,2] * 100, 2)

    data_vr[n_ind-1,1] = 100
    data_vr[n_ind-1,3] = 100
    cl = []

    for j in range(n_ind):
        cl.append(f'{data_vr[j,7]} - {data_vr[j,8]}')

    try: name_g = name_group_all[TARGET]    
    except: name_g = data[TARGET].unique()

    df = {f'{name_g[0]} (абс.)': [int(data_vr[j,0]) for j in range(n_ind)], 
          f'{name_g[0]} (%)': [data_vr[j,1] for j in range(n_ind)],
          f'{name_g[1]} (абс.)' : [int(data_vr[j,2]) for j in range(n_ind)], 
          f'{name_g[1]} (%)': [data_vr[j,3] for j in range(n_ind)],
          'F': [data_vr[j,4] for j in range(n_ind)],
          'p': [data_vr[j,5] for j in range(n_ind)],
          'OR': [data_vr[j,6] for j in range(n_ind)],
          'CI - 95%': [cl[j] for j in range(n_ind)],

         } 

    df = pd.DataFrame(df)

    df.loc[n_ind-1, 'F' : 'CI - 95%'] = ''
    
    try:
        name = name_group_all[i]
        name.append('Всего:')
        df.index = name
    except: 1

    display(df)  

In [231]:
for i in categorial:
    print(i)
    test_fishera(data, i, name_group_all)

SNP_rs896


,БАТТ (абс.),БАТТ (%),БАСТ (абс.),БАСТ (%),F,p,OR,CI - 95%
TT,10,20.83,16,28.57,0.66,0.658,0.66,0.27 - 1.63
TC,28,58.33,24,42.86,1.87,1.867,1.87,0.86 - 4.08
CC,10,20.83,16,28.57,0.66,0.658,0.66,0.27 - 1.63
Всего:,48,100.00,56,100.00,,,,


SNP_rs2072100


,БАТТ (абс.),БАТТ (%),БАСТ (абс.),БАСТ (%),F,p,OR,CI - 95%
0,34,70.83,24,42.86,3.24,3.238,3.24,1.43 - 7.33
1,10,20.83,14,25.00,0.79,0.789,0.79,0.31 - 1.99
2,4,8.33,18,32.14,0.19,0.192,0.19,0.06 - 0.61
3,48,100.00,56,100.00,,,,


SNP_rs3771829


,БАТТ (абс.),БАТТ (%),БАСТ (абс.),БАСТ (%),F,p,OR,CI - 95%
0,46,95.83,48,85.71,3.83,3.833,3.83,0.77 - 19.0
1,2,4.17,8,14.29,0.26,0.261,0.26,0.05 - 1.29
2,48,100.00,56,100.00,,,,


SNP_rs3823082


,БАТТ (абс.),БАТТ (%),БАСТ (абс.),БАСТ (%),F,p,OR,CI - 95%
0,30,62.50,36,66.67,0.83,0.833,0.83,0.37 - 1.87
1,10,20.83,8,14.81,1.51,1.513,1.51,0.54 - 4.21
2,8,16.67,10,18.52,0.88,0.88,0.88,0.32 - 2.45
3,48,100.00,54,100.00,,,,


Фенотип


,БАТТ (абс.),БАТТ (%),БАСТ (абс.),БАСТ (%),F,p,OR,CI - 95%
0,32,40.0,8,8.51,7.17,7.167,7.17,3.06 - 16.8
1,32,40.0,36,38.30,1.07,1.074,1.07,0.58 - 1.97
2,16,20.0,50,53.19,0.22,0.22,0.22,0.11 - 0.43
3,80,100.0,94,100.00,,,,


In [173]:
data[(data['SNP_rs896'] == 1) & (data[TARGET] == 3)][TARGET].count()

28

## print(st.fisher_exact(data))